In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kaggle-llm-science-exam/sample_submission.csv
/kaggle/input/kaggle-llm-science-exam/train.csv
/kaggle/input/kaggle-llm-science-exam/test.csv
/kaggle/input/additional-train-data-for-llm-science-exam/6000_train_examples.csv
/kaggle/input/additional-train-data-for-llm-science-exam/extra_train_set.csv
/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/saved_model.pb
/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/keras_metadata.pb
/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/assets/vocab.txt
/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/variables/variables.index
/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/variables/variables.data-00000-of-00001
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/saved_model.pb
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/keras_metadata.pb
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/assets/vocab.txt
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12

**SETTING**

In [2]:
import keras
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
import random
from sklearn.model_selection import KFold
from tensorflow.keras.layers import concatenate

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
LABEL_DICT = {"A":0,'B':1,'C':2,'D':3,'E':4}

In [4]:
def read_data(path,check_test= False):
    df = pd.read_csv(path[0])
    if not check_test:
        lst = ['prompt','A','B','C','D','E','answer']
        df = df[lst]
        for i in path[1:]:
            newdf = pd.read_csv(i)[lst]
            df = df.append(newdf)
        df.reset_index(drop=True, inplace=True)
    X_Prompt = tf.convert_to_tensor(df.prompt.to_list()) 
    X_A = tf.convert_to_tensor(df.A.to_list())
    X_B = tf.convert_to_tensor(df.B.to_list())
    X_C = tf.convert_to_tensor(df.C.to_list())
    X_D = tf.convert_to_tensor(df.D.to_list())
    X_E = tf.convert_to_tensor(df.E.to_list())  
    
    Selects = {'A':X_A,"B":X_B,"C":X_C,'D':X_D,'E':X_E}
    
    Labels = labels_one_hot = None
    
    
    if not check_test:
        labels = df.answer.to_list()
        labels_one_hot = [LABEL_DICT.get(i) for i in labels]

        label_A = []
        label_B = []
        label_C = []
        label_D = []
        label_E = []

        for i in labels:
            if i =='A':
                label_A.append(0.8)
                label_B.append(0.05)
                label_C.append(0.05)
                label_D.append(0.05)  
                label_E.append(0.05)
            if i =='B':
                label_B.append(0.8)
                label_A.append(0.05)
                label_C.append(0.05)
                label_D.append(0.05)  
                label_E.append(0.05)

            if i =='C':
                label_C.append(0.8)
                label_A.append(0.05)
                label_B.append(0.05)
                label_D.append(0.05)  
                label_E.append(0.05)
            if i =='D':
                label_D.append(0.8)
                label_A.append(0.05)
                label_B.append(0.05)
                label_C.append(0.05) 
                label_E.append(0.05)
            if i =='E':
                label_E.append(0.8)
                label_A.append(0.05)
                label_B.append(0.05)
                label_C.append(0.05)
                label_D.append(0.05)  
        label_A = tf.convert_to_tensor(label_A)
        label_B = tf.convert_to_tensor(label_B)
        label_C = tf.convert_to_tensor(label_C)
        label_D = tf.convert_to_tensor(label_D)
        label_E = tf.convert_to_tensor(label_E)


        Labels  = {'A': label_A,'B': label_B,'C': label_C,'D': label_D,'E': label_E}
    
    return {'Prompt':X_Prompt,'Selects':Selects,'labels':Labels,'label_onehot':labels_one_hot}

**INPUT TRAIN**

*4 thread model*

*Binary model*

In [5]:
path_train = ['/kaggle/input/additional-train-data-for-llm-science-exam/extra_train_set.csv', '/kaggle/input/additional-train-data-for-llm-science-exam/6000_train_examples.csv']


In [6]:
result = read_data(path_train,False)
X_Prompt = result.get('Prompt') 
X_A = result.get('Selects').get('A')
X_B = result.get('Selects').get('B')
X_C = result.get('Selects').get('C')
X_D = result.get('Selects').get('D')
X_E = result.get('Selects').get('E')

/tmp/ipykernel_26/3473268617.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(newdf)


In [7]:
Selects = [result.get('Selects')[i] for i in result.get('Selects') ]

In [8]:
Selects.insert(0,X_Prompt)

In [9]:
labels = [result.get('labels')[i] for i in result.get('labels') ]
labels_one_hot = result.get('label_onehot')
labels_one_hot_train = tf.one_hot(labels_one_hot, depth=5)

In [10]:
path_val = ['/kaggle/input/kaggle-llm-science-exam/train.csv']
result_val = read_data(path_val ,False)
X_Prompt_val  = result_val .get('Prompt') 
X_A_val  = result_val .get('Selects').get('A')
X_B_val  = result_val .get('Selects').get('B')
X_C_val  = result_val .get('Selects').get('C')
X_D_val  = result_val .get('Selects').get('D')
X_E_val  = result_val .get('Selects').get('E')

Selects_val  = [result_val .get('Selects')[i] for i in result_val .get('Selects') ]
Selects_val .insert(0,X_Prompt_val )
labels_val  = [result_val .get('labels')[i] for i in result_val .get('labels') ]
labels_one = result_val .get('label_onehot')
labels_one_hot_val = tf.one_hot(labels_one, depth=5)

**LOAD MODEL BERT**

In [11]:
bert_preprocess = hub.KerasLayer("/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3")
bert_encoder = hub.KerasLayer("/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2")

**BUILD MODEL**

In [12]:
text_Prompt = tf.keras.layers.Input(shape=(), dtype=tf.string)
text_A = tf.keras.layers.Input(shape=(), dtype=tf.string)
text_B = tf.keras.layers.Input(shape=(), dtype=tf.string)
text_C = tf.keras.layers.Input(shape=(), dtype=tf.string)
text_D = tf.keras.layers.Input(shape=(), dtype=tf.string)
text_E = tf.keras.layers.Input(shape=(), dtype=tf.string)

encoder_prompt_input = bert_preprocess(text_Prompt)
encoder_A_input = bert_preprocess(text_A)
encoder_B_input = bert_preprocess(text_B)
encoder_C_input = bert_preprocess(text_C)
encoder_D_input = bert_preprocess(text_D)
encoder_E_input = bert_preprocess(text_E)

In [13]:
encoder_prompt = concatenate(
        tuple([bert_encoder(encoder_prompt_input)['encoder_outputs'][i] for i in range(-4, 0)]), 
        name = 'hidden_states_prompt_1',
        axis = -1
    )[:, 0, :]
encoder_A = concatenate(
        tuple([bert_encoder(encoder_A_input)['encoder_outputs'][i] for i in range(-4, 0)]), 
        name = 'hidden_states_A',
        axis = -1
    )[:, 0, :]
encoder_B = concatenate(
        tuple([bert_encoder(encoder_B_input)['encoder_outputs'][i] for i in range(-4, 0)]), 
        name = 'hidden_states_B',
        axis = -1
    )[:, 0, :]
encoder_C = concatenate(
        tuple([bert_encoder(encoder_C_input)['encoder_outputs'][i] for i in range(-4, 0)]), 
        name = 'hidden_states_C',
        axis = -1
    )[:, 0, :]
encoder_D = concatenate(
        tuple([bert_encoder(encoder_D_input)['encoder_outputs'][i] for i in range(-4, 0)]), 
        name = 'hidden_states_D',
        axis = -1
    )[:, 0, :]
encoder_E = concatenate(
        tuple([bert_encoder(encoder_E_input)['encoder_outputs'][i] for i in range(-4, 0)]), 
        name = 'hidden_states_E',
        axis = -1
    )[:, 0, :]
print(encoder_E.shape)

reshape_prompt = tf.reshape(encoder_prompt, (-1, 4, 768))
reshape_A = tf.reshape(encoder_A, (-1, 4, 768))
reshape_B = tf.reshape(encoder_B, (-1, 4, 768))
reshape_C = tf.reshape(encoder_C, (-1, 4, 768))
reshape_D = tf.reshape(encoder_D, (-1, 4, 768))
reshape_E = tf.reshape(encoder_E, (-1, 4, 768))
print('reshape: ', reshape_A.shape)
# layer A
concat_encoder_A = concatenate([reshape_prompt,reshape_A,reshape_B,reshape_C,reshape_D,reshape_E],axis = 1)
LSTM = tf.keras.layers.LSTM(units=32,dropout=0.2, recurrent_dropout=0.2,return_sequences=True)(concat_encoder_A)
Dense_prompt_A = tf.keras.layers.Dense(32,activation = 'relu')(LSTM)
Dense_prompt_A =  tf.keras.layers.Dropout(0.3)(Dense_prompt_A)
Dense_prompt_A = tf.keras.layers.LayerNormalization()(Dense_prompt_A)
Linear_A =  tf.keras.layers.Dropout(0.3)(Dense_prompt_A)
classifer = keras.layers.Flatten()(Linear_A)
classifer = tf.keras.layers.Dense(5,activation = 'linear')(classifer)
classifer =  tf.keras.layers.Dense(5,activation = 'softmax')(classifer)

model = keras.Model(inputs=[text_Prompt, text_A,text_B,text_C,text_D,text_E], outputs=classifer)
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])
model.summary()

(None, 3072)
reshape:  (None, 4, 768)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None,)]            0           []                               
                                                        

In [14]:
EarlyStopping = tf.keras.callbacks.EarlyStopping(
                    monitor="val_loss",
                    patience=10,
                    restore_best_weights=True,
                )

In [15]:
# print(Selects[0][0])
# print('A')
# print(Selects[1][0])
# print('B')
# print(Selects[2][0])
# print('C')
# print(Selects[3][0])
# print('D')
# print(Selects[4][0])
# print('E')
# print(Selects[5][0])


In [16]:
model.fit(x = Selects,y = labels_one_hot_train,batch_size=32,
          epochs=50,  steps_per_epoch=60,
         validation_data=(Selects_val, labels_one_hot_val) , callbacks=[EarlyStopping])# 50 epoch

Epoch 1/50
60/60 [==============================] - 296s 4s/step - loss: 2.1714 - accuracy: 0.1911 - val_loss: 1.6938 - val_accuracy: 0.1850
Epoch 2/50
60/60 [==============================] - 230s 4s/step - loss: 1.9337 - accuracy: 0.2042 - val_loss: 1.6061 - val_accuracy: 0.2700
Epoch 3/50
60/60 [==============================] - 230s 4s/step - loss: 1.7803 - accuracy: 0.1964 - val_loss: 1.5970 - val_accuracy: 0.2350
Epoch 4/50
60/60 [==============================] - 226s 4s/step - loss: 1.7085 - accuracy: 0.1977 - val_loss: 1.5982 - val_accuracy: 0.2150
Epoch 5/50
60/60 [==============================] - 229s 4s/step - loss: 1.6769 - accuracy: 0.2104 - val_loss: 1.6053 - val_accuracy: 0.2150
Epoch 6/50
60/60 [==============================] - 229s 4s/step - loss: 1.6546 - accuracy: 0.1927 - val_loss: 1.6139 - val_accuracy: 0.2200
Epoch 7/50
60/60 [==============================] - 226s 4s/step - loss: 1.6525 - accuracy: 0.1845 - val_loss: 1.6187 - val_accuracy: 0.1850
Epoch 8/50
60

In [17]:
# model.save('19_epoch_13.h5')

In [18]:
# y_predict = model.predict(Selects)
# top_three_indices = (-y_predict).argsort(axis = 1)[:, :3].tolist()
# top_max = np.argmax(y_predict,axis = -1)
# report = classification_report(labels_one_hot, top_max)
# print(report)

In [19]:
# count = 0
# for i in range(len(labels_one_hot)):
#     if labels_one_hot[i] in top_three_indices[i]:
#         count +=1
# print(count)

In [20]:
y_predict = model.predict(Selects_val)
top_three_indices = (-y_predict).argsort(axis = 1)[:, :3].tolist()
top_max = np.argmax(y_predict,axis = -1)
report = classification_report(labels_one, top_max)
print(report)

7/7 [==============================] - 37s 3s/step
              precision    recall  f1-score   support

           0       0.14      0.08      0.10        37
           1       0.30      0.48      0.37        48
           2       0.22      0.48      0.30        44
           3       0.00      0.00      0.00        38
           4       0.00      0.00      0.00        33

    accuracy                           0.23       200
   macro avg       0.13      0.21      0.15       200
weighted avg       0.15      0.23      0.17       200



In [21]:
count = 0
for i in range(len(labels_one)):
    if labels_one[i] in top_three_indices[i]:
        count +=1
print(count)

133


**DATA TEST**

In [22]:
path_test = ['/kaggle/input/kaggle-llm-science-exam/test.csv']
result = read_data(path_test,True)
X_Prompt = result.get('Prompt') 
X_A = result.get('Selects').get('A')
X_B = result.get('Selects').get('B')
X_C = result.get('Selects').get('C')
X_D = result.get('Selects').get('D')
X_E = result.get('Selects').get('E')

Selects = [result.get('Selects')[i] for i in result.get('Selects') ]
Selects.insert(0,X_Prompt)

In [23]:
y_predict = model.predict(Selects)
top_three_indices = (-y_predict).argsort(axis = 1)[:, :3].tolist()

7/7 [==============================] - 21s 3s/step


In [24]:
# Mở tệp CSV để ghi
df_resutl =  pd.read_csv("/kaggle/input/kaggle-llm-science-exam/sample_submission.csv")
x = ['A','B','C','D',"E"]
Label_test = [' '.join([x[int(j)] for j in i]) for i in top_three_indices]

for i in range(len(Label_test)):
    df_resutl.at[i, 'prediction'] = str(Label_test[i])
df_resutl.to_csv('/kaggle/working/submission.csv', index=False)